In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torch transformers

In [11]:
import torch
import torchaudio
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import os
import torch.nn.functional as F
from tqdm import tqdm

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [9]:
entity_audio_dir = "/kaggle/input/business-json/entity_aud_files/kaggle/working/entity_aud_files"
dataset_audio_dir = '/kaggle/input/business-json/aud_files/kaggle/working/TTS/aud_files'

In [6]:
# Load Wav2Vec2
wav2vec_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
wav2vec_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

# Load ImageBind (hypothetical code, replace with the actual model)
# imagebind_model = ImageBindModel.from_pretrained("path_to_imagebind_model")


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# Function to truncate audio to 3.5 seconds
def truncate_audio(audio, sample_rate, duration=3.5):
    max_length = int(duration * sample_rate)
    if audio.size(1) > max_length:
        audio = audio[:, :max_length]
    return audio

# Function to get Wav2Vec2 embeddings
def get_wav2vec2_embeddings(audio, processor, model, max_length):
    audio = audio.squeeze(0)
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True, truncation=True,max_length = max_length)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings


# # Function to get ImageBind embeddings (hypothetical)
# def get_imagebind_embeddings(audio, model):
#     # Replace with actual ImageBind model code
#     embeddings = model.get_audio_embeddings(audio)
#     return embeddings

# Function to calculate cosine similarity
def cosine_similarity(emb1, emb2):
    return F.cosine_similarity(emb1, emb2)

# Function to rank entities based on similarity
def rank_entities(dataset_embeddings, entity_embeddings):
    rankings = {}
    for audio_filename, audio_emb in dataset_embeddings.items():
        similarities = []
        for entity_filename, entity_emb in entity_embeddings.items():
            similarity = cosine_similarity(audio_emb, entity_emb)
            similarities.append((entity_filename, similarity.item()))
        
        rankings[audio_filename] = sorted(similarities, key=lambda x: x[1], reverse=True)
    return rankings


# Function to calculate Top-N accuracy
def calculate_top_n_accuracy(rankings, sentence_df, entity_df, n=1):
    correct = 0
    total = len(sentence_df)

    for idx, row in sentence_df.iterrows():
        sentence_audio = row['audio_filename']
        # Check if 'entities' is a list of dictionaries and contains at least one entity
        if isinstance(row['entities'], list) and len(row['entities']) > 0:
            actual_entity_name = row['entities'][0]['mention']
            actual_entity = entity_map.get(actual_entity_name, None)
            # If actual_entity is not found in entity_map, skip this entry
            if actual_entity is None:
                continue

            # Get top-n ranked entities for this sentence
            top_n = [entity for entity, _ in rankings.get(sentence_audio, [])[:n]]
            # Check if the actual entity is within the top-n predictions
            if actual_entity in top_n:
                correct += 1

    return correct / total


In [16]:
# Process entity audio files
entity_embeddings_wav2vec = {}
# entity_embeddings_imagebind = {}

file_list = os.listdir(entity_audio_dir)

for filename in tqdm(file_list, desc="Processing audio files", unit="file"):
    try:
        file_path = os.path.join(entity_audio_dir, filename)
        audio, sample_rate = torchaudio.load(file_path)
        audio = truncate_audio(audio, sample_rate)
        max_length = int(3.5 * 16000) 
        wav2vec_embeddings = get_wav2vec2_embeddings(audio, wav2vec_processor, wav2vec_model, max_length)
        # imagebind_embeddings = get_imagebind_embeddings(audio, imagebind_model)
        
        entity_embeddings_wav2vec[filename] = wav2vec_embeddings
        # entity_embeddings_imagebind[filename] = imagebind_embeddings
    except Exception as e:
        print(f"Error processing {filename}: {e}")

Processing audio files: 100%|██████████| 500/500 [03:56<00:00,  2.12file/s]


In [17]:
# Process dataset audio files
dataset_embeddings_wav2vec = {}
# dataset_embeddings_imagebind = {}
file_list = os.listdir(dataset_audio_dir)
for filename in tqdm(file_list, desc="Processing audio files", unit="file"):
    try:
        file_path = os.path.join(dataset_audio_dir, filename)
        audio, sample_rate = torchaudio.load(file_path)
        max_length = (3*16000)
        wav2vec_embeddings = get_wav2vec2_embeddings(audio, wav2vec_processor, wav2vec_model, max_length)
    #     imagebind_embeddings = get_imagebind_embeddings(audio, imagebind_model)

        dataset_embeddings_wav2vec[filename] = wav2vec_embeddings
    #     dataset_embeddings_imagebind[filename] = imagebind_embeddings
    except Exception as e:
        print(f"Error processing {filename}: {e}")


Processing audio files: 100%|██████████| 2626/2626 [35:26<00:00,  1.23file/s]


In [29]:
import ast


sentence_df = pd.read_csv('/kaggle/input/business-json/Test (1).csv') 
entity_df = pd.read_csv('/kaggle/input/business-json/entity_aud_files.csv')

# Convert 'entities' column from string to list of dictionaries
sentence_df['entities'] = sentence_df['entities'].apply(lambda x: ast.literal_eval(x))

# Map entity_name to ID for easy lookup
entity_map = dict(zip(entity_df['entity_name'], entity_df['ID']))

In [ ]:
# Rank entities for Wav2Vec2 and ImageBind embeddings
rankings_wav2vec = rank_entities(dataset_embeddings_wav2vec, entity_embeddings_wav2vec)

In [35]:

# Calculate Top-N accuracies for Wav2Vec2
top1_acc_wav2vec = calculate_top_n_accuracy(rankings_wav2vec, sentence_df, entity_df, n=1)
top5_acc_wav2vec = calculate_top_n_accuracy(rankings_wav2vec, sentence_df, entity_df, n=5)
top10_acc_wav2vec = calculate_top_n_accuracy(rankings_wav2vec, sentence_df, entity_df, n=10)

# Print accuracy results
print(f"Wav2Vec2 Top-1 Accuracy: {top1_acc_wav2vec:.2f}")
print(f"Wav2Vec2 Top-5 Accuracy: {top5_acc_wav2vec:.2f}")
print(f"Wav2Vec2 Top-10 Accuracy: {top10_acc_wav2vec:.2f}")


Wav2Vec2 Top-1 Accuracy: 0.00
Wav2Vec2 Top-5 Accuracy: 0.00
Wav2Vec2 Top-10 Accuracy: 0.00


In [ ]:

# rankings_imagebind = rank_entities(dataset_embeddings_imagebind, entity_embeddings_imagebind)


# top1_acc_imagebind = top_n_accuracy(rankings_imagebind, entity_embeddings_imagebind.keys(), n=1)
# top5_acc_imagebind = top_n_accuracy(rankings_imagebind, entity_embeddings_imagebind.keys(), n=5)
# top10_acc_imagebind = top_n_accuracy(rankings_imagebind, entity_embeddings_imagebind.keys(), n=10)

# print(f"ImageBind Top-1 Accuracy: {top1_acc_imagebind:.2f}")
# print(f"ImageBind Top-5 Accuracy: {top5_acc_imagebind:.2f}")
# print(f"ImageBind Top-10 Accuracy: {top10_acc_imagebind:.2f}")